In [1]:
import numpy as np
import matplotlib as plt
import scipy as sc
import sumo as sm
import os
from traffic_initialization import *
import sys
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'

In [ ]:
# initial xml encoding
minGap=3
inflow=500

filedir = './xml_data/test1/'

netfile = 'test1.net.xml'
routefile = 'test1.rou.xml'
configfile = 'test1.sumocfg'
outpufile = 'test1.statistic.output.xml'

rtFl = RouteFile(filedir + routefile)

vtype = 'id0'

rtFl.setVehicleType(vtype, {
'accel' : "3.0",
'decel' : "6.0",
'length' : "4.0",
'minGap' : str(minGap),
'maxSpeed' : "50.0",
'sigma' : "0.5"
})

rtFl.setVehicleFlow(vtype, source='E0', dest='E3', num=inflow, dur=100, speed=13.89)
rtFl.save()

generateConfigFile(
filedir + configfile, 
netfile,
routefile,
output=['statistic', 'tripinfo']
)

loadConfig(filedir +'test1.sumocfg')

# waiting?

In [ ]:
# xml reading

f_name = filedir + outpufile

with open(f_name, 'r') as file:
    data = file.read()

data = BeautifulSoup(data, "xml")
ans = float(data('vehicleTripStatistics')[0]['duration'])
print(float(data('performance')[0]['duration']))

In [2]:
ans

NameError: name 'ans' is not defined

In [18]:
def loss(minGap, road, inflow):

    # initial xml encoding
    
    
    filedir = './xml_data/test1/'
    netfile = 'test1.net.xml'
    routefile = 'test1.rou.xml'
    configfile = 'test1.sumocfg'
    outpufile = 'test1.statistic.output.xml'
    
    rtFl = RouteFile(filedir + routefile)
    
    vtype = 'id0'
    
    rtFl.setVehicleType(vtype, {
    'accel' : "3.0",
    'decel' : "6.0",
    'length' : "4.0",
    'minGap' : str(minGap),
    'maxSpeed' : "50.0",
    'sigma' : "0.5"
    })
    
    rtFl.setVehicleFlow(vtype, source='E0', dest='E3', num=inflow, dur=100, speed=13.89)
    rtFl.save()
    
    generateConfigFile(
    filedir + configfile, 
    netfile, 
    routefile,
    output=['statistic']#, 'tripinfo']
    )
    
    loadConfig(filedir +'test1.sumocfg')
    
    # waiting?
    
    # xml reading

    f_name = filedir + outpufile

    with open(f_name, 'r') as file:
        data = file.read()

    data = BeautifulSoup(data, "xml")
    ans = float(data('vehicleTripStatistics')[0]['duration'])

    return ans

In [6]:
loss(minGap=3, road='road_1', inflow=200)

Step #259.00 (0ms ?*RT. ?UPS, vehicles TOT 200 ACT 0 BUF 0)                               


IndexError: list index out of range

In [ ]:
# graphs

X = np.asarray([])
Y = np.zeros(X.shape[0])

for i in range(X.shape[0]):
    
    Y[i] = loss(X[i], '', inflow)

plt.plot(X, Y)

In [ ]:
# simple optimization

bounds = (, )

sol = sc.optimize.dual_annealing(loss, bounds=[bounds], args=['road', inflow], maxiter=200)
ans = sol.x

In [ ]:
# smart ml optimization?